# 1. Import Libaries

In [1]:
pip install spotipy sqlalchemy

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 5.3 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.0/252.0 KB 9.1 MB/s eta 0:00:000:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 616.0/616.0 KB 32.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
import spotipy
import requests
from time import sleep

# 2. input client id and client secret

In [2]:
# Prompt the user to enter their Spotify API credentials
#client_id = input('Enter your Spotify client ID: ')
#client_secret = input('Enter your Spotify client secret: ')

In [3]:
client_id = 'dc71185ea84d41929498146825dc3477'
client_secret = 'ec172fd80bf4460abcf1b0b7eb83766c'

# 3. pull data

In [4]:
# Get an access token from the Spotify API
response = requests.post('https://accounts.spotify.com/api/token', data={'grant_type': 'client_credentials'}, auth=(client_id, client_secret))
access_token = response.json()['access_token']

In [5]:
# Set up the headers for the HTTP GET request
headers = {
    'Authorization': f'Bearer {access_token}',
    'Content-Type': 'application/json',
}

In [6]:
# url is just Today's Top Hits (Top 50 songs today on April 24)
urls = ['https://open.spotify.com/playlist/37i9dQZF1DXcBWIGoYBM5M?si=8f130f7a0a7f4ebb']

year = [2024]
# for i in range(2000,2023):
 #   year.append(i)

playlists_api = []
for url in urls:
    id = url.split("/")[-1].split("?")[0]
    api = 'https://api.spotify.com/v1/playlists/' + id + '/tracks'
    playlists_api.append(api)
playlists_api = pd.Series(playlists_api, index = year)

In [7]:
TopHits = pd.DataFrame(columns=['playlist_url',
                                'year',
                                'track_id',
                                'track_name', 
                                'track_popularity', 
                                'album',
                                'artist_id',
                                'artist_name', 
                                'artist_genres', 
                                'artist_popularity',  
                                'danceability', 
                                'energy', 
                                'key', 
                                'loudness', 
                                'mode', 
                                'speechiness', 
                                'acousticness', 
                                'instrumentalness', 
                                'liveness', 
                                'valence', 
                                'tempo', 
                                'duration_ms', 
                                'time_signature'])

In [9]:
for playlist_api in playlists_api:
    response = requests.get(playlist_api, headers=headers)
    top_tracks = response.json()['items']
    playlist_url = 'https://open.spotify.com/playlist/' + playlist_api.split("/")[-2].split("?")[0]
    year = playlists_api.index[playlists_api == playlist_api][0]
    for top_track in top_tracks:        
        track_id = top_track['track']['uri'].replace('spotify:track:', '')
        artist_id = top_track['track']['artists'][0]['uri'].replace('spotify:artist:', '')
        artist_info = requests.get('https://api.spotify.com/v1/' + 'artists' + '/' + artist_id, headers=headers)
        artist_info = artist_info.json()
        audio_features = requests.get('https://api.spotify.com/v1/' + 'audio-features' + '/' + track_id, headers=headers)
        
        if audio_features.status_code == 404:
            new_row = pd.Series({'track_id': track_id,
                   'track_name': top_track['track']['name'], 
                   'track_popularity': top_track['track']['popularity'], 
                   'album': top_track['track']['album']['name'],
                   'artist_id': artist_id,
                   'artist_name': top_track['track']['artists'][0]['name'], 
                   'artist_genres': artist_info['genres'], 
                   'artist_popularity': artist_info['popularity']})
        else:
            audio_features = audio_features.json()
            new_row = pd.Series({'track_id': track_id,
                   'track_name': top_track['track']['name'], 
                   'track_popularity': top_track['track']['popularity'], 
                   'album': top_track['track']['album']['name'],
                   'artist_id': artist_id,
                   'artist_name': top_track['track']['artists'][0]['name'], 
                   'artist_genres': artist_info['genres'], 
                   'artist_popularity': artist_info['popularity'],  
                   'danceability': audio_features['danceability'], 
                   'energy': audio_features['energy'], 
                   'key': audio_features['key'], 
                   'loudness': audio_features['loudness'], 
                   'mode': audio_features['mode'], 
                   'speechiness': audio_features['speechiness'], 
                   'acousticness': audio_features['acousticness'], 
                   'instrumentalness': audio_features['instrumentalness'], 
                   'liveness': audio_features['liveness'], 
                   'valence': audio_features['valence'], 
                   'tempo': audio_features['tempo'], 
                   'duration_ms': audio_features['duration_ms'], 
                   'time_signature': audio_features['time_signature']})
        TopHits = pd.concat([TopHits, new_row.to_frame().T], ignore_index=True)
        sleep(0.2)
    values = {'playlist_url': playlist_url, 'year': year}
    TopHits = TopHits.fillna(value=values)

In [14]:
audio_features = requests.get('https://api.spotify.com/v1/' + 'audio-features' + '/' + '37i9dQZEVXbMDoHDwVN2tF', headers=headers)

In [15]:
audio_features

<Response [404]>

In [12]:
TopHits

,playlist_url,year,track_id,track_name,track_popularity,album,artist_id,artist_name,artist_genres,artist_popularity,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,https://open.spotify.com/playlist/37i9dQZF1DXc...,2024.0,2OzhQlSqBEmt7hmkYxfT6m,Fortnight (feat. Post Malone),89,THE TORTURED POETS DEPARTMENT,06HL4z0CvFAxyc27GXpf02,Taylor Swift,[pop],100,...,-10.895,1,0.0245,0.499,0.000006,0.0939,0.319,95.988,228965,4
1,https://open.spotify.com/playlist/37i9dQZF1DXc...,2024.0,4IadxL6BUymXlh8RCJJu7T,Too Sweet,90,Unheard,2FXC3k01G6Gw61bmprjgqS,Hozier,"[irish singer-songwriter, modern rock, pov: in...",85,...,-5.505,1,0.0412,0.0295,0.000809,0.0398,0.934,117.038,251424,4
2,https://open.spotify.com/playlist/37i9dQZF1DXc...,2024.0,2GxrNKugF82CnoRFbQfzPf,i like the way you kiss me,100,i like the way you kiss me,0PCCGZ0wGLizHt2KZ7hhA2,Artemas,[],81,...,-4.263,1,0.0447,0.000938,0.0106,0.0826,0.747,151.647,142515,4
3,https://open.spotify.com/playlist/37i9dQZF1DXc...,2024.0,2qSkIjg1o9h3YT9RAgYN75,Espresso,92,Espresso,74KM79TiuVKeVCqs8QtB0B,Sabrina Carpenter,[pop],81,...,-5.486,1,0.0293,0.133,0.000012,0.197,0.661,103.982,175459,4
4,https://open.spotify.com/playlist/37i9dQZF1DXc...,2024.0,3rUGC1vUpkDG9CZFHMur1t,greedy,97,greedy,45dkTj5sMRSjrmBSBeiHym,Tate McRae,[pop],83,...,-3.18,0,0.0319,0.256,0,0.114,0.844,111.018,131872,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,https://open.spotify.com/playlist/37i9dQZF1DXc...,2024.0,42VsgItocQwOQC3XWZ8JNA,FE!N (feat. Playboi Carti),93,UTOPIA,0Y5tJX1MQlPlqiwlOH1tJY,Travis Scott,"[hip hop, rap, slap house]",91,...,-2.777,0,0.06,0.0316,0,0.142,0.201,148.038,191701,4
96,https://open.spotify.com/playlist/37i9dQZF1DXc...,2024.0,4o120XeV8els1S5bu7mzBX,Eyes Closed,83,Eyes Closed,53XhwfbYqKCa1cC15pYq2q,Imagine Dragons,"[modern rock, pop, rock]",85,...,-7.199,0,0.114,0.0323,0.0014,0.0732,0.512,173.913,200010,4
97,https://open.spotify.com/playlist/37i9dQZF1DXc...,2024.0,2H2ytI7ompfNmRkI2sq8Uk,I LUV IT (feat. Playboi Carti),86,I LUV IT (feat. Playboi Carti),4nDoRrQiYLoBzwC5BhVJzF,Camila Cabello,"[dance pop, pop]",78,...,-5.764,1,0.0467,0.0149,0.000394,0.227,0.228,151.01,174886,4
98,https://open.spotify.com/playlist/37i9dQZF1DXc...,2024.0,2KslE17cAJNHTsI2MI0jb2,Standing Next to You,92,GOLDEN,6HaGTQPmzraVmaVxvz6EUc,Jung Kook,[k-pop],84,...,-4.389,0,0.0955,0.0447,0,0.339,0.816,106.017,206020,4


In [16]:
TopHits.to_csv('playlist_2024.csv',index=False)